In [ ]:
# !pip install optuna

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc, warnings, random, time, os

from pathlib import Path

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns

import gc
gc.enable()

import optuna

### Folders and Dataframes

In [2]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [3]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [4]:
def remove_unnecessary(df):
    df.drop(df[df['target'] == 0].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
remove_unnecessary(train_df)

In [5]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2828,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2829,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2830,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2831,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Config and Seeding

In [6]:
class Config(): 
    NUM_FOLDS = 6
    NUM_EPOCHS = 3
    BATCH_SIZE = 16
    MAX_LEN = 248
    EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
    MODEL_PATH = 'google/electra-large-discriminator'
    TOKENIZER_PATH = 'google/electra-large-discriminator'
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    SEED = 1000
    NUM_WORKERS = 2
    MODEL_FOLDER = MODELS_PATH
    model_name = 'electra-large-discriminator'
    svm_kernels = ['rbf']
    svm_c = 5

cfg = Config()

In [7]:
if not cfg.MODEL_FOLDER.exists():
    os.mkdir(cfg.MODEL_FOLDER)

In [8]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

### Dataset

In [9]:
def add_bins(train_df, num_bins):
    train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
    return num_bins

In [10]:
add_bins(train_df, cfg.NUM_FOLDS)

6

In [11]:
train_df.groupby(['bins'])['target'].agg(['count', 'mean'])

,count,mean
bins,,
0,122,-3.125765
1,441,-2.270279
2,784,-1.412150
3,886,-0.548095
4,494,0.289716
5,106,1.070237


In [12]:
tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)

In [13]:
class CommonLitDataset(Dataset):
    def __init__(self, df, tokenizer, inference_only=False):
        super().__init__()
        self.df, self.inference_only = df, inference_only
        self.text = df['excerpt'].tolist()
        self.bins = df['bins']
        if not inference_only:
            self.target = torch.tensor(df['target'].to_numpy(), dtype = torch.float32)
        
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',
            max_length = cfg.MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )
        
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return {'input_ids': input_ids, 'attention_mask': attention_mask}
        else:
            target = self.target[index]
            return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target': target}
    
    def __len__(self):
        return len(self.df)

In [14]:
sample_ds = CommonLitDataset(train_df, tokenizer)

### Model

In [15]:
class AttentionHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [16]:
from transformers import AutoModelForSequenceClassification

class CommonLitModel(nn.Module):
    def __init__(self):
        super(CommonLitModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.MODEL_PATH)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModelForSequenceClassification.from_pretrained(cfg.MODEL_PATH, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['hidden_states']
        last_layer_hidden_states = hidden_states[-1]
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector

In [17]:
sample_model = CommonLitModel()

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier

In [18]:
import re

for i, (name, param) in enumerate(sample_model.named_parameters()):
    if(name.find('layer') > -1):
        layer_name = re.sub(r'.+(layer\.\d+).+', r'\1', name)

In [19]:
for i, (name, param) in enumerate(sample_model.named_parameters()):
    print(i, name, param.size())

0 transformer_model.electra.embeddings.word_embeddings.weight torch.Size([30522, 1024])
1 transformer_model.electra.embeddings.position_embeddings.weight torch.Size([512, 1024])
2 transformer_model.electra.embeddings.token_type_embeddings.weight torch.Size([2, 1024])
3 transformer_model.electra.embeddings.LayerNorm.weight torch.Size([1024])
4 transformer_model.electra.embeddings.LayerNorm.bias torch.Size([1024])
5 transformer_model.electra.encoder.layer.0.attention.self.query.weight torch.Size([1024, 1024])
6 transformer_model.electra.encoder.layer.0.attention.self.query.bias torch.Size([1024])
7 transformer_model.electra.encoder.layer.0.attention.self.key.weight torch.Size([1024, 1024])
8 transformer_model.electra.encoder.layer.0.attention.self.key.bias torch.Size([1024])
9 transformer_model.electra.encoder.layer.0.attention.self.value.weight torch.Size([1024, 1024])
10 transformer_model.electra.encoder.layer.0.attention.self.value.bias torch.Size([1024])
11 transformer_model.electra.

In [20]:
sample_input_ids = torch.randint(0, 1000, [8, 248])
sample_attention_mask = torch.randint(0, 1000, [8, 248])

In [21]:
sample_model(sample_input_ids, sample_attention_mask)[1].shape

torch.Size([8, 1024])

In [22]:
torch.sum(torch.randn([8, 496, 768]), axis=1)

tensor([[-15.8633,  15.8719,  32.3934,  ...,   1.7552, -21.7107,   6.4586],
        [ -8.0905, -10.3193,   9.1525,  ...,   2.1436,  15.2303, -46.2511],
        [-21.1742, -22.8150,  -2.1771,  ...,  29.3484, -25.6957, -13.4151],
        ...,
        [ 31.1003, -14.0406, -20.5553,  ..., -16.9740, -21.2865, -15.3540],
        [  7.0861,   4.3567,  13.9963,  ...,  -1.2864,  -5.4169,   3.2112],
        [ -1.5488,  14.2157,  27.7912,  ...,   5.4052,  -4.6318,  46.8670]])

### Evaluation and Prediction

In [23]:
def eval_mse(model, data_loader):
    model.eval()
    mse_sum = 0
    mse_loss = nn.MSELoss(reduction='sum')
    
    with torch.no_grad():
        for batch_num, record in enumerate(data_loader):
            input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            mse_sum += mse_loss(pred.flatten().cpu(), target.cpu())
            
    return mse_sum / len(data_loader.dataset)

In [24]:
def predict(model, data_loader):
    model.eval()
    result = []
    
    with torch.no_grad():
        for batch_num, record in tqdm(enumerate(data_loader), total=len(data_loader)):
            input_ids, attention_mask = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            result.extend(pred.flatten().to("cpu").tolist())
            
    return np.array(result)

In [25]:
sample_dl = DataLoader(sample_ds, shuffle=False, batch_size=16, num_workers=1)

### Optimizer and Sampler

In [26]:
5e-5 / 2.5, 5e-5 / 0.5, 5e-5

(2e-05, 0.0001, 5e-05)

In [27]:
def create_optimizer(model, base_lr=5e-5, last_lr=None):
    named_parameters = list(model.named_parameters())
    
    regressor_param_start = 397
    attention_param_start = 393
    roberta_parameters = named_parameters[:attention_param_start]
    attention_parameters = named_parameters[attention_param_start:regressor_param_start]
    regressor_parameters = named_parameters[regressor_param_start:]
    
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]
    
    parameters = []
    if last_lr is not None:
        parameters.append({"params": attention_group, "lr": last_lr})
        parameters.append({"params": regressor_group, "lr": last_lr})
    else:
        parameters.append({"params": attention_group})
        parameters.append({"params": regressor_group})
        
    # Change on different models
    layer_low_threshold = 133
    layer_middle_threshold = 261
    
    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if 'bias' in name else 0.01
        
        lr = base_lr / 2.5 # 2e-05
        if layer_num >= layer_middle_threshold:
            lr = base_lr / 0.5 # 1e-4
        elif layer_num >= layer_low_threshold:        
            lr = base_lr    
            
        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})
        
    return AdamW(parameters)

In [28]:
sample_optimizer = create_optimizer(sample_model)

In [29]:
from torch.utils.data import Sampler,SequentialSampler,RandomSampler,SubsetRandomSampler
from collections import Counter

class WeightedSampler(Sampler):
    
    def __init__(self, dataset):
        
        self.indices = list(range(len(dataset)))
        self.num_samples = len(dataset)
        self.label_to_count = dict(Counter(dataset.bins))
        weights = [1/self.label_to_count[i] for i in dataset.bins]
        
        self.weights = torch.tensor(weights,dtype=torch.double)
        
    def __iter__(self):
        count = 0
        index = [self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True)]
        while count < self.num_samples:
            yield index[count]
            count += 1
    
    def __len__(self):
        return self.num_samples

### Training

In [30]:
def choose_eval_period(val_rmse):
    for rmse, period in cfg.EVAL_SCHEDULE:
        if val_rmse >= rmse:
            return period

In [31]:
def serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, model_path):
    if not best_val_rmse or val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch = epoch
        if not model_path.parent.exists():
            os.makedirs(model_path.parent)
        
        torch.save(model.state_dict(), model_path)
        print(f"New best_val_rmse: {best_val_rmse:0.4}")
    else:       
        print(f"Still best_val_rmse: {best_val_rmse:0.4}",
              f"(from epoch {best_epoch})")
    return best_epoch, best_val_rmse

In [32]:
class Trainer():
    def __init__(self, scaler, model, model_path, train_loader, val_loader, optimizer, scheduler=None, num_epochs=cfg.NUM_EPOCHS):
        self.scaler, self.model, self.model_path, self.train_loader, self.val_loader, self.optimizer, self.scheduler, self.num_epochs = (
            scaler, model, model_path, train_loader, val_loader, optimizer, scheduler, num_epochs
        )
        self.mse_loss = nn.MSELoss(reduction='mean')
        
    def train_loss_backward(self, input_ids, attention_mask, target):
        self.optimizer.zero_grad()
        if self.scaler == None:
            pred, _ = self.model(input_ids, attention_mask)
            mse = self.mse_loss(pred.flatten(), target)
            mse.backward()
            self.optimizer.step()
            self.scheduler.step()
        else:
            with torch.cuda.amp.autocast():
                pred, _ = self.model(input_ids, attention_mask)
                mse = self.mse_loss(pred.flatten(), target)
            self.scaler.scale(mse).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
            
    def train(self):
        self.model.train()
        
        best_val_rmse = None
        best_epoch = 0
        step = 0
        last_eval_step = 0
        eval_period = cfg.EVAL_SCHEDULE[0][1]    

        start = time.time()
        val_rmse_list = []
        
        tbar = tqdm(range(self.num_epochs), total=self.num_epochs)
        for epoch in tbar:
            tbar.set_description(f'Epoch: {epoch}')
            val_rmse = None
            for batch_num, record in enumerate(self.train_loader):
                input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
                
                self.train_loss_backward(input_ids, attention_mask, target)
                    
                if step >= last_eval_step + eval_period:
                    elapsed_seconds = time.time() - start
                    num_steps = step - last_eval_step
                    print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                    last_eval_step = step
                    
                    val_rmse = np.sqrt(eval_mse(self.model, self.val_loader))
                    print(f"Epoch: {epoch} batch_num: {batch_num}", f"val_rmse: {val_rmse:0.4} ", end='')
                    
                    eval_period = choose_eval_period(val_rmse)
                    best_epoch, best_val_rmse = serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, self.model, self.model_path)
                    val_rmse_list.append(val_rmse)
                    start = time.time()
                # Finish early on condition
                if epoch > 0 and best_val_rmse > 0.6 or (len(val_rmse_list) > 5 and np.array(val_rmse_list).mean() > 1.0):
                    return best_val_rmse
                
                step += 1
        return best_val_rmse

In [33]:
kfold = KFold(n_splits=cfg.NUM_FOLDS, random_state=cfg.SEED, shuffle=True)
splits = list(kfold.split(train_df))

### Main Training

In [34]:
def train_fold(base_lr, last_lr, fold = 0):
    
    print(f'##### Using fold {fold}')
    print(f'##### Using base_lr {base_lr} last_lr {last_lr}')
    
    model_path = cfg.MODEL_FOLDER/f"{cfg.model_name.replace('/', '_')}_{fold + 1}/model_{fold + 1}.pth"
    
    set_random_seed(cfg.SEED + fold)
    
    tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)
    
    train_indices, val_indices = splits[fold]
    train_dataset = CommonLitDataset(train_df.loc[train_indices], tokenizer)    
    val_dataset = CommonLitDataset(train_df.loc[val_indices], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE,
                              drop_last=False, shuffle=True, num_workers=cfg.NUM_WORKERS)    
    val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=cfg.NUM_WORKERS)
    
    set_random_seed(cfg.SEED + fold)
    
    model = CommonLitModel().to(cfg.DEVICE)
    
    optimizer = create_optimizer(model, base_lr=base_lr, last_lr=last_lr)
    
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_training_steps=cfg.NUM_EPOCHS * len(train_loader), 
                                                num_warmup_steps=50)
    
    scaler = torch.cuda.amp.GradScaler()
    scaler = None
    
    trainer = Trainer(scaler, model, model_path, train_loader, val_loader, optimizer, scheduler = scheduler)
    rmse_val = trainer.train()
    tokenizer.save_pretrained(str(model_path.parent))
    
    return rmse_val

In [35]:
# Best results
# Fold 0: {'base_lr': 6.155021772017101e-05, 'last_lr': 0.004642225106260296}. Best is trial 11 with value: 0.4787840247154236
# Fold 1: {'base_lr': 8.902912488113375e-05, 'last_lr': 0.00023076670834499074}. Best is trial 9 with value: 0.447449654340744
# Fold 2: {'base_lr': 0.00010637579365513648, 'last_lr': 8.276647346442369e-05}. Best is trial 9 with value: 0.47160205245018005
# Fold 3: {'base_lr': 3.543430147790451e-05, 'last_lr': 0.00011412924670673832}. Best is trial 6 with value: 0.46940940618515015
# Fold 4: {'base_lr': 4.8817697487830015e-05, 'last_lr': 0.00010249423984922014}. Best is trial 1 with value: 0.48146629333496094
# Fold 5: {'base_lr': 8.26551556012735e-05, 'last_lr': 0.00012486898084560538}. Best is trial 1 with value: 0.47751927375793457

In [36]:
lr_list = [
    {'base_lr': 6.155021772017101e-05, 'last_lr': 0.004642225106260296},
    {'base_lr': 8.902912488113375e-05, 'last_lr': 0.00023076670834499074},
    {'base_lr': 0.00010637579365513648, 'last_lr': 8.276647346442369e-05},
    {'base_lr': 3.543430147790451e-05, 'last_lr': 0.00011412924670673832},
    {'base_lr': 4.8817697487830015e-05, 'last_lr': 0.00010249423984922014}, 
    {'base_lr': 8.26551556012735e-05, 'last_lr': 0.00012486898084560538}
]

In [37]:
%%time

rmse_values = []
for i in range(len(list(splits))):
    fold = i
    lrs = lr_list[fold]
    rmse_val = train_fold(lrs['base_lr'], lrs['last_lr'], fold=fold)
    print(f'Final RMSE: {rmse_val}')
    rmse_values.append(rmse_val)

##### Using fold 0
##### Using base_lr 6.155021772017101e-05 last_lr 0.004642225106260296


Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier


16 steps took 13.7 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9218 New best_val_rmse: 0.9218

16 steps took 11.9 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.9639 Still best_val_rmse: 0.9218 (from epoch 0)

16 steps took 12.0 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7486 New best_val_rmse: 0.7486

16 steps took 12.0 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6446 New best_val_rmse: 0.6446

16 steps took 12.0 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6073 New best_val_rmse: 0.6073

16 steps took 12.0 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6321 Still best_val_rmse: 0.6073 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5637 New best_val_rmse: 0.5637

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5644 Still best_val_rmse: 0.5637 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.655 Still best_val_rmse: 0.5637 (from epoch 0)

16 steps took 12.3 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5554 New best_val_rms

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier


16 steps took 13.3 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.7079 New best_val_rmse: 0.7079

16 steps took 12.0 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6837 New best_val_rmse: 0.6837

16 steps took 12.0 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.9618 Still best_val_rmse: 0.6837 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6293 New best_val_rmse: 0.6293

16 steps took 12.1 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6916 Still best_val_rmse: 0.6293 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5629 New best_val_rmse: 0.5629

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6269 Still best_val_rmse: 0.5629 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5378 New best_val_rmse: 0.5378

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5174 New best_val_rmse: 0.5174

16 steps took 12.3 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5937 Still best_val_rmse: 0.5174 (fro

...


16 steps took 13.3 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.082 New best_val_rmse: 1.082

16 steps took 12.0 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7597 New best_val_rmse: 0.7597

16 steps took 12.0 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7525 New best_val_rmse: 0.7525

16 steps took 12.0 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6422 New best_val_rmse: 0.6422

16 steps took 12.1 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5748 New best_val_rmse: 0.5748

16 steps took 12.1 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.7174 Still best_val_rmse: 0.5748 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5421 New best_val_rmse: 0.5421

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5537 Still best_val_rmse: 0.5421 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5463 Still best_val_rmse: 0.5421 (from epoch 0)

16 steps took 12.2 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5767 Still best_val_rmse: 0.5421 (from 

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier


16 steps took 13.3 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9823 New best_val_rmse: 0.9823

16 steps took 12.0 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.8517 New best_val_rmse: 0.8517

16 steps took 12.0 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6583 New best_val_rmse: 0.6583

16 steps took 12.1 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6434 New best_val_rmse: 0.6434

16 steps took 12.1 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5579 New best_val_rmse: 0.5579

16 steps took 12.1 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5827 Still best_val_rmse: 0.5579 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.7169 Still best_val_rmse: 0.5579 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6163 Still best_val_rmse: 0.5579 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5809 Still best_val_rmse: 0.5579 (from epoch 0)

16 steps took 12.3 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5231 New best_val_rm

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier


16 steps took 13.2 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9287 New best_val_rmse: 0.9287

16 steps took 12.0 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7149 New best_val_rmse: 0.7149

16 steps took 12.0 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7342 Still best_val_rmse: 0.7149 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6573 New best_val_rmse: 0.6573

16 steps took 12.1 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.7195 Still best_val_rmse: 0.6573 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5843 New best_val_rmse: 0.5843

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6044 Still best_val_rmse: 0.5843 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6077 Still best_val_rmse: 0.5843 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5619 New best_val_rmse: 0.5619

16 steps took 12.2 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.558 New best_val_rms

In [38]:
f'mean RMSE values: {np.mean(np.array(rmse_values))}'

'mean RMSE values: 0.47103846073150635'

### Verify the model

In [39]:
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [40]:
cfg.model_offset = 0
cfg.model_limit = 6
cfg.n_folds = 5
cfg.svm_kernels = ['rbf']
cfg.svm_c = 5

In [41]:
num_bins = int(np.ceil(np.log2(len(train_df))))
train_df['bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
bins = train_df['bins'].values

In [42]:
%%time

inference_models = []
for i in range(1, cfg.NUM_FOLDS + 1):
    print(f'Model {i}')
    inference_model = CommonLitModel()
    inference_model = inference_model.cuda()
    inference_model.load_state_dict(torch.load(str(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}/model_{i}.pth")))
    inference_model.eval();
    inference_models.append(inference_model)

Model 1


Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier

Model 2


Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier

Model 3


...

Model 5


Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier

Model 6


Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier

CPU times: user 22.3 s, sys: 7.1 s, total: 29.4 s
Wall time: 39.5 s


In [44]:
from transformers import ElectraTokenizer

tokenizers = []
for i in range(1, cfg.NUM_FOLDS):
    tokenizer = ElectraTokenizer.from_pretrained(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}")
    tokenizers.append(tokenizer)

In [45]:
def get_cls_embeddings(dl, transformer_model):
    cls_embeddings = []
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            output, context_vector = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda())
#             cls_embeddings.extend(output['last_hidden_state'][:,0,:].detach().cpu().numpy())
            embedding_out = context_vector.detach().cpu().numpy()
            cls_embeddings.extend(embedding_out)
    return np.array(cls_embeddings)

In [46]:
def rmse_score(X, y):
    return np.sqrt(mean_squared_error(X, y))

In [47]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, test_id, tokenizer, max_len=128):
        self.excerpt = text
        self.test_id = test_id
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return {'input_ids': convert_to_list(encode['input_ids']),
                'attention_mask': convert_to_list(encode['attention_mask']),
                'id': self.test_id[idx]}
    
    def __len__(self):
        return len(self.excerpt)

In [48]:
def create_dl(df, tokenizer):
    text = df['excerpt'].values
    ids = df['id'].values
    ds = CommonLitDataset(text, ids, tokenizer, max_len=cfg.MAX_LEN)
    return DataLoader(ds, 
                      batch_size = cfg.BATCH_SIZE,
                      shuffle=False,
                      num_workers = 1,
                      pin_memory=True,
                      drop_last=False
                     )

In [49]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
remove_unnecessary(train_df)

In [50]:
train_target_mean = train_df['target'].mean()
train_target_std = train_df['target'].std()
train_df['normalized_target'] = (train_df['target'] - train_target_mean) / train_target_std

In [51]:
%%time

train_target = train_df['normalized_target'].values

def calc_mean(scores):
    return np.mean(np.array(scores), axis=0)

final_scores = []
final_rmse = []
kernel_rmse_score_mean = []
final_kernel_predictions_means = []
for j, (inference_model, tokenizer) in enumerate(zip(inference_models, tokenizers)):
    print('Model', j)
    test_dl = create_dl(test_df, tokenizer)
    train_dl = create_dl(train_df, tokenizer)
    transformer_model = inference_model
    transformer_model.cuda()
    X = get_cls_embeddings(train_dl, transformer_model)
    
    y = train_target
    X_test = get_cls_embeddings(test_dl, transformer_model)
    
    kfold = StratifiedKFold(n_splits=cfg.NUM_FOLDS)
    scores = []
    rmse_scores = []
    kernel_predictions_means = []
    for kernel in cfg.svm_kernels:
        print('Kernel', kernel)
        kernel_scores = []
        kernel_rmse_scores = []
        kernel_predictions = []
        for k, (train_idx, valid_idx) in enumerate(kfold.split(X, bins)):

            print('Fold', k, train_idx.shape, valid_idx.shape)
            model = SVR(C=cfg.svm_c, kernel=kernel, gamma='auto')

            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_valid)
            kernel_predictions.append(prediction)
            kernel_rmse_scores.append(rmse_score(prediction, y_valid))
            print('rmse_score', kernel_rmse_scores[k])
            kernel_scores.append(model.predict(X_test))
        kernel_predictions_means.append(np.array([np.mean(kp) for kp in kernel_predictions]).mean())
        scores.append(calc_mean(kernel_scores))
        kernel_rmse_score = calc_mean(kernel_rmse_scores)
        kernel_rmse_score_mean.append(kernel_rmse_score)
        rmse_scores.append(kernel_rmse_score)
    final_kernel_predictions_means.append(kernel_predictions_means)
    final_scores.append(calc_mean(scores))
    final_rmse.append(calc_mean(rmse_scores))
print('FINAL RMSE score', np.mean(np.array(final_rmse)))

Model 0



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3047332220488328
Fold 1 (2361,) (472,)
rmse_score 0.29730153563307293
Fold 2 (2361,) (472,)
rmse_score 0.2957023622448626
Fold 3 (2361,) (472,)
rmse_score 0.27814858052977703
Fold 4 (2361,) (472,)
rmse_score 0.30497148036370575
Fold 5 (2361,) (472,)
rmse_score 0.31681242524116554
Model 1



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.2920565856752555
Fold 1 (2361,) (472,)
rmse_score 0.31450678049627995
Fold 2 (2361,) (472,)
rmse_score 0.29631847712899523
Fold 3 (2361,) (472,)
rmse_score 0.29395091994887385
Fold 4 (2361,) (472,)
rmse_score 0.2961161707304299
Fold 5 (2361,) (472,)
rmse_score 0.31105137736329896
Model 2



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.30843181018529525
Fold 1 (2361,) (472,)
rmse_score 0.3073181217298621
Fold 2 (2361,) (472,)
rmse_score 0.3089643106431168
Fold 3 (2361,) (472,)
rmse_score 0.31473145569485866
Fold 4 (2361,) (472,)
rmse_score 0.3186755752138472
Fold 5 (2361,) (472,)
rmse_score 0.3168873004667106
Model 3



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3070133372663501
Fold 1 (2361,) (472,)
rmse_score 0.31807177728626657
Fold 2 (2361,) (472,)
rmse_score 0.31992449804970824
Fold 3 (2361,) (472,)
rmse_score 0.29759807285099893
Fold 4 (2361,) (472,)
rmse_score 0.3171389958203484
Fold 5 (2361,) (472,)
rmse_score 0.3205302029286147
Model 4



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3009744605957655
Fold 1 (2361,) (472,)
rmse_score 0.27362538143516296
Fold 2 (2361,) (472,)
rmse_score 0.2602109284224828
Fold 3 (2361,) (472,)
rmse_score 0.28097097098875384
Fold 4 (2361,) (472,)
rmse_score 0.3006269887840996
Fold 5 (2361,) (472,)
rmse_score 0.29813239042029627
FINAL RMSE score 0.3023832165395696
CPU times: user 4min 56s, sys: 4.53 s, total: 5min 1s
Wall time: 4min 59s


In [52]:
final_kernel_predictions_means

[[0.0009605669676436582],
 [-0.004022977014025861],
 [0.0006062137297803185],
 [-0.0012854642062001438],
 [-0.0011118016136740696]]

In [53]:
# (train_df['target'] - cfg.train_target_mean) / cfg.train_target_std
final_scores_normalized = np.array(final_scores) * train_target_std + train_target_mean

In [54]:
kernel_rmse_score_mean_array = np.array(kernel_rmse_score_mean)
kernel_rmse_score_mean_sum = np.sum(kernel_rmse_score_mean_array)
prop_losses = kernel_rmse_score_mean_array / kernel_rmse_score_mean_sum
prop_losses_sum = (1 - prop_losses).sum()
weights = (1 - prop_losses) / prop_losses_sum
weights

array([0.2004583 , 0.20028383, 0.19832692, 0.19818173, 0.20274922])

In [55]:
def calc_mean(scores, weights=weights):
    return np.average(np.array(scores), weights=weights, axis=0)

In [56]:
target_mean = train_df['target'].mean()
final_scores_flat = calc_mean(final_scores_normalized).flatten()
final_scores_mean = final_scores_flat.mean()
target_mean, np.array(final_scores_normalized).mean()
# (-0.9579984513405823, -0.8029817438292849)

(-0.9596573929279916, -0.9967448270735803)

In [57]:
final_scores_flat

array([-0.42098746, -0.71595805, -0.53248514, -2.23908212, -1.85958253,
       -1.31527597,  0.10749485])

In [58]:
mean_diff = target_mean - final_scores_mean
mean_diff, mean_diff / len(final_scores)

(0.03689638105009252, 0.007379276210018504)

In [59]:
sample_df['target'] = final_scores_flat + mean_diff
# sample_df['target'] = len(final_scores) / np.sum(1 / np.array(final_scores), axis=0) # harmonic mean
sample_df

,id,target
0,c0f722661,-0.384091
1,f0953f0a5,-0.679062
2,0df072751,-0.495589
3,04caf4e0c,-2.202186
4,0e63f8bea,-1.822686
5,12537fe78,-1.278380
6,965e592c0,0.144391


### Prepare Packaging

In [60]:
cfg.model_name

'electra-large-discriminator'

In [61]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [62]:
BEST_MODEL_FOLDER

PosixPath('/home/commonlit/models/electra-large-discriminator/best')

In [63]:
cfg.NUM_FOLDS

6

In [64]:
bestmodels = [MODELS_PATH/f'{cfg.model_name}_{i + 1}' for i in range(0, cfg.NUM_FOLDS)]

In [65]:
bestmodels

[PosixPath('/home/commonlit/models/electra-large-discriminator_1'),
 PosixPath('/home/commonlit/models/electra-large-discriminator_2'),
 PosixPath('/home/commonlit/models/electra-large-discriminator_3'),
 PosixPath('/home/commonlit/models/electra-large-discriminator_4'),
 PosixPath('/home/commonlit/models/electra-large-discriminator_5'),
 PosixPath('/home/commonlit/models/electra-large-discriminator_6')]

In [68]:
from shutil import copyfile

def normalize_name(path_name):
    return path_name.replace('', '')

for i, best_model in enumerate(bestmodels):
    print(f'Processing {i}th model')
    i = i + 1
    best_model_file = f'{best_model}/model_{i}.pth'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/tokenizer_config.json'))
        assert tokenizer_json.exists(), f'{tokenizer_json} does not exist'
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/tokenizer.json'))
        assert vocab_txt.exists(), f'{vocab_txt} does not exist'
        copyfile(vocab_txt, tokenizer_path/'tokenizer.json')

        special_tokens = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/special_tokens_map.json'))
        assert special_tokens.exists()
        copyfile(special_tokens, tokenizer_path/'special_tokens_map')
        
        spiece_model = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/vocab.txt'))
        assert spiece_model.exists()
        copyfile(spiece_model, tokenizer_path/'vocab.txt')
    else:
        print(f'{best_model_file} is missing')

Processing 0th model
Processing 1th model
Processing 2th model
Processing 3th model
Processing 4th model
Processing 5th model


In [69]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/electra-large-discriminator/best_models.zip'

In [70]:
!ls {MODELS_PATH/cfg.model_name}

best  best_models.zip


In [71]:
!mv {MODELS_PATH}/{cfg.model_name}.yaml {MODELS_PATH/cfg.model_name}

mv: cannot stat '/home/commonlit/models/electra-large-discriminator.yaml': No such file or directory


In [72]:
transformer_model.transformer_model.save_pretrained(save_directory=f'{MODELS_PATH/cfg.model_name}/lm')

In [73]:
!du -h {MODELS_PATH/cfg.model_name}/*

1.2M	/home/commonlit/models/electra-large-discriminator/best/tokenizer-1
688K	/home/commonlit/models/electra-large-discriminator/best/tokenizer-2
688K	/home/commonlit/models/electra-large-discriminator/best/tokenizer-3
688K	/home/commonlit/models/electra-large-discriminator/best/tokenizer-4
688K	/home/commonlit/models/electra-large-discriminator/best/tokenizer-5
688K	/home/commonlit/models/electra-large-discriminator/best/tokenizer-6
7.6G	/home/commonlit/models/electra-large-discriminator/best
7.0G	/home/commonlit/models/electra-large-discriminator/best_models.zip
1.3G	/home/commonlit/models/electra-large-discriminator/lm


In [74]:
shutil.make_archive(MODELS_PATH/cfg.model_name/'lm', 'zip', f'{MODELS_PATH/cfg.model_name}/lm')

'/home/commonlit/models/electra-large-discriminator/lm.zip'

In [75]:
!kaggle datasets init -p {MODELS_PATH/cfg.model_name}

Data package template written to: /home/commonlit/models/electra-large-discriminator/dataset-metadata.json


In [76]:
dataset_json_path = Path(MODELS_PATH/cfg.model_name/'dataset-metadata.json')
assert dataset_json_path.exists()

In [77]:
!cat {str(dataset_json_path)}

{
  "title": "INSERT_TITLE_HERE",
  "id": "gilfernandes/INSERT_SLUG_HERE",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [78]:
with open(dataset_json_path, 'r') as f:
    dataset_json = f.read()
    dataset_json = dataset_json.replace('INSERT_TITLE_HERE', f'commonlit-{cfg.model_name}').replace('INSERT_SLUG_HERE', f'commonlit-{cfg.model_name}')
    print(dataset_json)
with(open(dataset_json_path, 'w')) as f:
    f.write(dataset_json)

{
  "title": "commonlit-electra-large-discriminator",
  "id": "gilfernandes/commonlit-electra-large-discriminator",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}


In [79]:
!rm -rf {MODELS_PATH/cfg.model_name}/best
!rm -rf {MODELS_PATH/cfg.model_name}/lm

In [80]:
!kaggle datasets create -p {MODELS_PATH/cfg.model_name}

Starting upload for file best_models.zip
100%|██████████████████████████████████████| 6.97G/6.97G [11:32<00:00, 10.8MB/s]
Upload successful: best_models.zip (7GB)
Starting upload for file lm.zip
100%|██████████████████████████████████████| 1.16G/1.16G [01:59<00:00, 10.4MB/s]
Upload successful: lm.zip (1GB)
Your private Dataset is being created. Please check progress at /api/v1/datasets/status//gilfernandes/commonlit-electra-large-discriminator


In [ ]:
!kaggle datasets version -p {MODELS_PATH/cfg.model_name} -m "Version with spiece.model" -d

In [ ]:
state_dict = torch.load(str(MODELS_PATH/f'distilroberta-0/checkpoint-105/pytorch_model.bin'))

In [ ]:
loaded_model = CommonLitModel()

In [ ]:
loaded_model.load_state_dict(state_dict)